In [1]:
import config # this is the api file
import pandas as pd
import numpy as np
import finnhub
import time
import datetime

In [2]:
#start and end date for the data
dt_start = "01/04/2021"
dt_end = "14/04/2022"

#convert the date data into unix timestamp
utc_start = int(datetime.datetime.strptime(dt_start, "%d/%m/%Y").timestamp())
utc_end = int(datetime.datetime.strptime(dt_end, "%d/%m/%Y").timestamp())

In [3]:
# list of stocks
stocks = ['AAPL','MSFT','OKTA', 'COST', 'KO', 'ABBV']

In [4]:
stk_df = pd.DataFrame(columns=['c','h','l','o','s','t','v','ticker'])

In [5]:
finnhub_client = finnhub.Client(api_key=config.finnhub_key)

def price_volume_data(stock):
    dict = (finnhub_client.stock_candles(stock, 'D', utc_start, utc_end))
    df = pd.DataFrame.from_dict(dict,orient='index').transpose()
    df['ticker'] = stock
    df['t'] = pd.to_datetime(df['t'], unit='s')
    return df

In [6]:
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

record_count = 1
set_count = 1

Beginning Data Retrieval     
-----------------------------


In [7]:
for i, stock in enumerate(stocks):
    if(i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(61) # the delay is required for finnhub's terms of service - 60 calls per minute - I've left it a 50
    
    # adding stock data to the dataframe
    try:
        stk_df = stk_df.append(price_volume_data(stock))
    
    # If an error is experienced, skip the city.
    except:
        print("Stock not found. Skipping...")
        pass
    
    # telling you what record is being processed
    print(f"Processing Record {record_count} of Set {set_count} | {stock}")
    
    # adding to the record count for the next iteration
    record_count += 1
    
# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Processing Record 1 of Set 1 | AAPL
Processing Record 2 of Set 1 | MSFT
Processing Record 3 of Set 1 | OKTA
Processing Record 4 of Set 1 | COST
Processing Record 5 of Set 1 | KO
Processing Record 6 of Set 1 | ABBV
-----------------------------
Data Retrieval Complete      
-----------------------------


In [8]:
stk_df = stk_df.drop(['s'], axis=1)

In [9]:
cols = {'c':'close',
       'h':'high',
       'l':'low',
       'o':'open',
       't':'date',
       'v':'volume'}

stk_df = stk_df.rename(columns=cols)

In [10]:
#number of stocks in the portfolio

num_assets = len(stocks)
num_trading_days = (pd.to_datetime(dt_end)-pd.to_datetime(dt_start)).days
num_random_portfolios = 25000

In [11]:
# format the dataframe tickers at top and date as index

stk_clsing_price_df = stk_df[['date','close','ticker']]
stk_clsing_price_df = stk_clsing_price_df.pivot(index='date',columns='ticker',values='close')

In [12]:
# daily security return

daily_sec_return = stk_clsing_price_df.pct_change()

In [13]:
# annualize the daily return

annual_sec_return = daily_sec_return.mean() * num_trading_days
annual_sec_return

ticker
AAPL    0.579795
ABBV    0.749081
COST    0.941302
KO      0.399527
MSFT    0.308544
OKTA   -0.582162
dtype: float64

In [14]:
# Calculate the covariance between stocks in the portfolio based on the daily return and then annualize the covariance
# by multiplying by the number of estimated trading days
# covariance will measure the degree that stocks included in the portfolio move in tandem
# negative covaried stocks move in opposite directions while positively covaried stocks move in the same direction

daily_sec_covariance = daily_sec_return.cov()
annual_sec_covariance = daily_sec_covariance * num_trading_days
annual_sec_covariance

ticker,AAPL,ABBV,COST,KO,MSFT,OKTA
ticker,,,,,,
AAPL,0.110777,0.012150,0.044234,0.016017,0.078954,0.095726
ABBV,0.012150,0.067876,0.013488,0.019934,0.014762,-0.010303
COST,0.044234,0.013488,0.075284,0.019684,0.049600,0.045475
KO,0.016017,0.019934,0.019684,0.040286,0.016341,-0.011965
MSFT,0.078954,0.014762,0.049600,0.016341,0.106829,0.122266
OKTA,0.095726,-0.010303,0.045475,-0.011965,0.122266,0.502034


In [15]:
# Annualize the daily std deviation
#
annual_sec_std_deviation = (daily_sec_return.std()) * np.sqrt(num_trading_days)
annual_sec_std_deviation

ticker
AAPL    0.332832
ABBV    0.260530
COST    0.274379
KO      0.200713
MSFT    0.326847
OKTA    0.708543
dtype: float64

In [16]:
# Generate an array where the number of rows equal num_random_portfolios and where the total number of columns will tie to the number of stocks in the portfolio
# the columns will total 1
#
X = np.random.dirichlet(np.ones(num_assets),size=num_random_portfolios)

In [17]:
# Convert the array to a dataframe and assign the ticker symbol to the column name
#
rand_port_wts = pd.DataFrame(X, columns=stk_clsing_price_df.columns)
rand_port_wts.shape

(25000, 6)

In [19]:
# multiply the random portfolio weights by the annualized return
#
portfolio_return = rand_port_wts.dot(annual_sec_return)
portfolio_return = pd.DataFrame(portfolio_return, columns=['portfolio_return'])
portfolio_return.shape

(25000, 1)

In [20]:
portfolio_std_deviation = pd.DataFrame(rand_port_wts.apply(lambda x: np.dot(x.T, np.sqrt(np.dot(annual_sec_covariance,x))),axis=1),columns=['portfolio_std_dev'])

C:\Users\ryanj\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in sqrt
  """Entry point for launching an IPython kernel.


In [21]:
portfolio_data = pd.DataFrame(rand_port_wts)
portfolio_data['std_dev'] = portfolio_std_deviation.portfolio_std_dev
portfolio_data['return'] = portfolio_return.portfolio_return
portfolio_data['sharpe_ratio'] = portfolio_data['return'] / portfolio_data['std_dev']
#portfolio_data

#portfolio_data.loc[portfolio_data.sharpe_ratio.idxmax()]
portfolio_data

ticker,AAPL,ABBV,COST,KO,MSFT,OKTA,std_dev,return,sharpe_ratio
0,0.014582,0.216801,0.038556,0.528269,0.179727,0.022065,0.173903,0.460815,2.649831
1,0.049511,0.379899,0.136988,0.010993,0.058624,0.363984,0.276577,0.252811,0.914071
2,0.094253,0.347702,0.004704,0.288249,0.025100,0.239992,0.208321,0.302725,1.453171
3,0.488705,0.291088,0.025080,0.039399,0.002198,0.153529,0.242966,0.452045,1.860532
4,0.229059,0.049867,0.139495,0.235190,0.288717,0.057671,0.219559,0.450942,2.053851
...,...,...,...,...,...,...,...,...,...
24995,0.061848,0.474722,0.258038,0.070773,0.053865,0.080755,0.193581,0.632239,3.266012
24996,0.217776,0.230481,0.171053,0.006574,0.346736,0.027379,0.228333,0.553598,2.424523
24997,0.064548,0.117547,0.188237,0.238348,0.343429,0.047891,0.204799,0.475973,2.324095
24998,0.232488,0.032282,0.075759,0.105967,0.207351,0.346153,0.320451,0.135086,0.421548
